<a href="https://colab.research.google.com/github/fmind/slides-to-translate/blob/main/Slides-to-Translate%20-%20Translate%20Google%20Slides%20Automatically%20with%20Gemini%20on%20Vertex%20AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Slides Translator with Vertex AI and Gemini

This notebook demonstrates how to automatically translate the text content of a Google Slides presentation into a target language using Vertex AI and the Gemini models.

**Prerequisites:**

*   A [Google Cloud Platform](https://console.cloud.google.com/) (GCP) project with the [Vertex AI](https://console.cloud.google.com/vertex-ai/studio/multimodal) enabled.
*   Sufficient permissions to access [Google Drive](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com) and [Google Slides](https://console.cloud.google.com/marketplace/product/google/slides.googleapis.com), and to use [Vertex AI](https://console.cloud.google.com/marketplace/product/google/aiplatform.googleapis.com).

**How to Use:**

1.  **Configure User Settings:** In the "User Configuration" cell, provide your GCP Project ID, the ID of the Google Slides presentation you want to translate, the target language, the desired Gemini model, the Vertex AI location, and the maximum number of workers for concurrent API calls.
2.  **Run the Notebook:** Execute all the code cells in sequence.
3.  **Access Translated Presentation:** Once the execution is complete, a link to the translated copy of your presentation will be provided in the output of the last code cell.

**What the notebook does:**

1.  Copies the original Google Slides presentation.
2.  Extracts all text elements from the copied presentation.
3.  Translates the extracted text using the specified Vertex AI Gemini model.
4.  Replaces the original text in the copied presentation with the translated text.
5.  Reports the token usage and estimated cost of the translation.

# Examples

- Original Slides: https://docs.google.com/presentation/d/1CvmBW0BmorHpC6NwYCh94qTAMZY9MlUx7ClcSwCg7Dg/edit?slide=id.p#slide=id.p
- Slides Translated in French:https://docs.google.com/presentation/d/1t7MqZG5BUCbTbDKlOu25e0BLi00IIIcFVWKZCCt8h2o/edit?slide=id.p#slide=id.p
- Slides Translated in German: https://docs.google.com/presentation/d/1jYGc7bqus-Kluvmww5U8T3WgALDayh5cfsYgJPiA6ds/edit?slide=id.p#slide=id.p
- Slides Translated in Luxembourgish: https://docs.google.com/presentation/d/1vcmMHp_vh1pMhNgLMlVyGC_wRJawtY9-_g8T1df5Hfo/edit

In [ ]:
# @title User Configuration
# @markdown Your Google Cloud project ID
PROJECT_ID = "slides-to-translate" # @param {type:"string"}
# @markdown The ID of your Google Slides presentation
PRESENTATION_ID = "1CvmBW0BmorHpC6NwYCh94qTAMZY9MlUx7ClcSwCg7Dg" # @param {type:"string"}
# @markdown The language you want to translate the text into
TARGET_LANGUAGE = "German" # @param {type:"string"}
# @markdown The AI model to use for translation
MODEL_NAME = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro"]
# @markdown The region for Vertex AI
LOCATION = "global" # @param {type:"string"}
# @markdown Number of workers for concurrent API calls
MAX_WORKERS = 10 # @param {type:"integer"}
# @markdown Extra prompt to guide the model output
EXTRA_PROMPT = "This is about a meetup resentation titled 'Strengthen your AI/ML Coding Skills with the MLOps Coding Course'" # @param {type:"string"}

In [ ]:
# @title Dependencies Installation
!pip install --quiet --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib google-genai

In [ ]:
# @title Initialization
import re
from google import genai
from google.colab import auth
from google.genai import types
from googleapiclient.discovery import build
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
# @title Authenticate with Google Cloud Project
auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
# @title Build the Google Drive and Slides service client
drive_service = build('drive', 'v3')
slides_service = build('slides', 'v1')

In [ ]:
# @title Build the Generative AI Client
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [ ]:
# @title Copy the original presentation

original_presentation = slides_service.presentations().get(presentationId=PRESENTATION_ID).execute()
original_title = original_presentation.get('title', 'Untitled')

print(f"📄 Making a copy of the presentation '{original_title}' ...")

copy_title = f"{original_title} ({TARGET_LANGUAGE} Translation)"
copied_file = drive_service.files().copy(fileId=PRESENTATION_ID, body={'name': copy_title}).execute()
copied_presentation_id = copied_file['id']
copied_presentation_url = f'https://docs.google.com/presentation/d/{copied_presentation_id}/edit'

print(f"✅ Successfully copied to: {copied_presentation_url}")

presentation = slides_service.presentations().get(presentationId=copied_presentation_id).execute()
slides = presentation.get('slides', [])
print(f"🔍 Found {len(slides)} slides in the new presentation.")

📄 Making a copy of the presentation 'Strengthen your AI/ML Coding Skills with the MLOps Coding Course - MLflow Ambassador - 2024-06-05' ...
✅ Successfully copied to: https://docs.google.com/presentation/d/1jYGc7bqus-Kluvmww5U8T3WgALDayh5cfsYgJPiA6ds/edit
🔍 Found 22 slides in the new presentation.


In [ ]:
# @title 2. Extract all text run elements

texts_to_locations = {}
for slide in slides:
    slide_id = slide['objectId']
    for element in slide.get('pageElements', []):
        if 'shape' in element and 'text' in element['shape']:
            for text_element in element['shape']['text'].get('textElements', []):
                if 'textRun' in text_element:
                    content = text_element['textRun']['content'].strip()
                    # Filter out empty strings and text without letters
                    if content and re.search('[a-zA-Z]', content):
                        if content not in texts_to_locations:
                            texts_to_locations[content] = set()
                        texts_to_locations[content].add(slide_id)

unique_texts = list(texts_to_locations.keys())
slides_ids = {slide_id for locations in texts_to_locations.values() for slide_id in locations}
print(f'🔍 Found {len(unique_texts)} individual text runs to translate across {len(slides_ids)} slides.')

🔍 Found 211 individual text runs to translate across 22 slides.


In [ ]:
# @title 3. Translating text to new language using Vertex AI

instructions = (
    f"Translate the following text to {TARGET_LANGUAGE} as accurately as possible. "
    f"You are provide with this context to perform the task: '{EXTRA_PROMPT}'"
    "Do not add any preamble, intro, or explanation; return only the translated text. "
)

usages = []
translations = {}

def translate_text(text):
    """Translates a single text string and returns the original and translated text with the model usage."""
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=text,
            config=types.GenerateContentConfig(
                system_instruction=instructions,
                temperature=0.0,
            )
        )
        translation = response.text.strip()
        usage = response.usage_metadata
        return text, translation, usage
    except Exception as error:
        print(f"Error translating '{text}': {error}")
        return text, None, None

# Use ThreadPoolExecutor for concurrent translation
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(translate_text, text) for text in unique_texts]
    for i, future in enumerate(as_completed(futures)):
        text, translation, usage = future.result()
        if usage:
            usages.append(usage)
        if translation:
            translations[text] = translation
        print(f"- [{i+1}/{len(futures)}] '{text}' -> '{translation}'")

- [1/211] 'Médéric HURIER & Matthieu JIMENEZ' -> 'Médéric HURIER und Matthieu JIMENEZ'
- [2/211] 'Dr. Médéric HURIER' -> 'Dr. Médéric HURIER'
- [3/211] 'About us' -> 'Über uns'
- [4/211] 'Decathlon Digital' -> 'Decathlon Digital'
- [5/211] 'Dr. Matthieu JIMENEZ' -> 'Dr. Matthieu JIMENEZ'
- [6/211] 'Freelance MLOps Engineer' -> 'Freiberuflicher MLOps-Ingenieur'
- [7/211] 'University of Luxembourg' -> 'Universität Luxemburg'
- [8/211] 'MLflow Ambassador - 2024-06-05' -> 'MLflow Ambassador - 2024-06-05'
- [9/211] 'https://www.fmind.dev/' -> 'https://www.fmind.dev/'
- [10/211] 'Where to begin ?' -> 'Wo anfangen?'
- [11/211] 'Strengthen your AI/ML Coding Skills with the' -> 'Stärken Sie Ihre KI/ML-Programmierkenntnisse mit dem'
- [12/211] 'Many things to know in Machine Learning (Frameworks, Solutions, Theories, Data …)' -> 'Vieles zu wissen im Maschinellen Lernen (Frameworks, Lösungen, Theorien, Daten …)'
- [13/211] 'Research Associate' -> 'Wissenschaftlicher Mitarbeiter'
- [14/211] 'Mad L

In [ ]:
# @title Preparing batch update requests

# Sort translations from longer to shorter strings to avoid conflicts
sorted_translations = sorted(translations.items(), key=lambda item: len(item[0]), reverse=True)

requests = []
for text, translation in sorted_translations:
    # Ensure the translated text is not empty or just whitespace
    if translation.strip():
        # Get the specific slide IDs for this text
        page_ids = texts_to_locations.get(text)
        if page_ids:
            requests.append({
                'replaceAllText': {
                    'replaceText': translation,
                    'pageObjectIds': list(page_ids),
                    'containsText': {
                        'text': text,
                        'matchCase': True,
                    }
                }
            })

print(f'🌟 Number of change requests: {len(requests)}')

🌟 Number of change requests: 211


In [ ]:
# @title Execute the batch update to replace all text

batch_size = 50
total_changes = 0
for i, start in enumerate(range(0, len(requests), batch_size)):
    stop = start + batch_size
    batch = requests[start:stop]
    body = {'requests': batch}
    print(f'- Batch {i}: {start} -> {stop}', end='')
    response = slides_service.presentations().batchUpdate(presentationId=copied_presentation_id, body=body).execute()
    changes = sum(reply.get('replaceAllText', {}).get('occurrencesChanged', 0) for reply in response.get('replies', []))
    print(f' - Changes: {changes}')
    total_changes += changes

print(f"✅ Success! Your presentation has been updated with {total_changes} changes:\n{copied_presentation_url}")

- Batch 0: 0 -> 50 - Changes: 50
- Batch 1: 50 -> 100 - Changes: 51
- Batch 2: 100 -> 150 - Changes: 50
- Batch 3: 150 -> 200 - Changes: 51
- Batch 4: 200 -> 250 - Changes: 11
✅ Success! Your presentation has been updated with 213 changes:
https://docs.google.com/presentation/d/1jYGc7bqus-Kluvmww5U8T3WgALDayh5cfsYgJPiA6ds/edit


In [ ]:
# @title 6. Report token usage and total cost

prices = {
    "gemini-2.5-pro": {
        "input": 1.25 / 1_000_000,
        "output": 10.00 / 1_000_000,
    },
    "gemini-2.5-flash": {
        "input": 0.30 / 1_000_000,
        "output": 2.50 / 1_000_000,
    },
    "gemini-2.5-flash-lite": {
        "input": 0.1 / 1_000_000,
        "output": 0.4 / 1_000_000,
    },
}

total_input_tokens = 0
total_output_tokens = 0
price_table = prices[MODEL_NAME]
price_input_tokens = price_table["input"]
price_output_tokens = price_table["output"]

for usage in usages:
    total_input_tokens += usage.prompt_token_count
    total_output_tokens += usage.candidates_token_count

input_cost = total_input_tokens * price_input_tokens
output_cost = total_output_tokens * price_output_tokens
total_cost = input_cost + output_cost

print(f"Total Input Tokens: {total_input_tokens}")
print(f"Total Output Tokens: {total_output_tokens}")
print(f"Input Tokens Cost ($): {input_cost}")
print(f"Total Output Cost ($): {output_cost}")
print(f"Total Cost ($): {total_cost}")

Total Input Tokens: 15088
Total Output Tokens: 1687
Input Tokens Cost ($): 0.0045264
Total Output Cost ($): 0.004217500000000001
Total Cost ($): 0.0087439
